# DS400: Assignment 5 (Python)

##### Tingyu Lin (Student ID: 5638679)


In [1]:
import requests
import numpy as np
import pandas as pd
import time

In [2]:
# hyperlink: https://github.com/ltyita/Python_Assign05.git

In [8]:
# Ex6 source API key from another script
with open("ticket_APIkey.py") as script:
    exec(script.readline())

### Exercise 7

In [11]:
from IPython.display import Markdown as md

# Exercise (7)
# first GET request that searches for event venues in Germany
response = requests.get("https://app.ticketmaster.com/discovery/v2/venues.json?",
                        params = {"countryCode": "DE", "apikey" : api_key})

# extract content
DE_content = response.json()


# number of page and observation
n = DE_content['page']
n


{'size': 20, 'totalElements': 4750, 'totalPages': 238, 'number': 0}

In [12]:
md(f'From data frame, it suggests that each page contains 20 observations,\
and there are {n['totalElements']} events in Germany, which make them {n["totalPages"]}-page long. And we are currently at page 0.')


SyntaxError: f-string: unmatched '[' (1680478580.py, line 2)